In [156]:
import matplotlib as plt
import pandas as pd
import os

def filter(df,column, value):
    indexNames = df[ (df[column] != value)].index
    df.drop(indexNames , inplace=True)
    df = df.reset_index(drop=True)
    return df

def getInfoFromFilename(file, fileNumber):
        tokenList = statFiles[fileNumber].split(' - ')
        tokenList[2] = tokenList[2][:-10]
        while len(tokenList) > 2:
            tokenList.pop(1)
        return tokenList

In [157]:
class Scenario:
    name = None
    scores = {}
    
    def __init__(self, name):
        self.name = name
    
    def addScoreFromFilename(self, timestamp, score):
        # TODO: filter out bad formatting
        self.scores[timestamp] = score

In [158]:
STATS_DIRECTORY = 'C:\ProgramData\SteamLibrary\steamapps\common\FPSAimTrainer\FPSAimTrainer\stats'

statFiles = []
for root, _, files in os.walk(STATS_DIRECTORY):
    for file in files:
        if file.endswith('.csv'):
            statFiles.append(file)
            
scenarioList = []

print(statFiles[218])

Ascended Tracking v3 - Challenge - 2020.05.17-20.07.35 Stats.csv


In [159]:
for fileNumber in range(0, len(statFiles)):
    df = pd.read_csv(STATS_DIRECTORY + '\\' +  statFiles[fileNumber], sep=',', error_bad_lines=False)
    
    df.rename(columns={'Kill #':'Headings'}, inplace=True)
    df.rename(columns={'Timestamp':'Values'}, inplace=True)

    df = df[['Headings','Values']]

    
    df = filter(df, 'Headings', 'Score:')
    
    name, timestamp = getInfoFromFilename(statFiles[fileNumber], fileNumber)
    
#     print("---------------------------------------------",name)
    scenarioNumber = -1
    index = 0
    for scenario in scenarioList:
        if scenario.name == name:
            scenarioNumber = index
            break
        index += 1

    if scenarioNumber == -1:
        scenarioList.append(Scenario(name))

#     print(scenarioNumber)
    try:
        scenarioList[scenarioNumber].addScoreFromFilename(timestamp, df["Values"].iloc[-1])
    except IndexError:
        print('Score unavailable for file: ', statFiles[fileNumber])

for scenario in scenarioList:
    print(scenario.name)
    for timestamp, score in scenario.scores.items():
        print(timestamp + " : " + score)
    print('\n\n\n')

b'Skipping line 92: expected 12 fields, saw 16\n'
b'Skipping line 85: expected 12 fields, saw 16\n'
b'Skipping line 84: expected 12 fields, saw 16\n'
b'Skipping line 66: expected 12 fields, saw 16\nSkipping line 67: expected 12 fields, saw 17\n'
b'Skipping line 68: expected 12 fields, saw 16\nSkipping line 69: expected 12 fields, saw 17\n'
b'Skipping line 65: expected 12 fields, saw 16\nSkipping line 66: expected 12 fields, saw 17\n'
b'Skipping line 66: expected 12 fields, saw 16\nSkipping line 67: expected 12 fields, saw 17\n'
b'Skipping line 64: expected 12 fields, saw 16\nSkipping line 65: expected 12 fields, saw 17\n'
b'Skipping line 69: expected 12 fields, saw 16\nSkipping line 70: expected 12 fields, saw 17\n'
b'Skipping line 67: expected 12 fields, saw 16\nSkipping line 68: expected 12 fields, saw 17\n'
b'Skipping line 62: expected 12 fields, saw 16\nSkipping line 63: expected 12 fields, saw 17\n'
b'Skipping line 66: expected 12 fields, saw 16\nSkipping line 67: expected 12 fiel

b'Skipping line 102: expected 12 fields, saw 16\n'
b'Skipping line 98: expected 12 fields, saw 16\n'
b'Skipping line 98: expected 12 fields, saw 16\n'
b'Skipping line 84: expected 12 fields, saw 16\n'
b'Skipping line 89: expected 12 fields, saw 16\n'
b'Skipping line 85: expected 12 fields, saw 16\n'
b'Skipping line 93: expected 12 fields, saw 16\n'
b'Skipping line 87: expected 12 fields, saw 16\n'
b'Skipping line 92: expected 12 fields, saw 16\n'
b'Skipping line 102: expected 12 fields, saw 16\n'
b'Skipping line 96: expected 12 fields, saw 16\n'
b'Skipping line 92: expected 12 fields, saw 16\n'
b'Skipping line 92: expected 12 fields, saw 16\n'
b'Skipping line 96: expected 12 fields, saw 16\n'
b'Skipping line 92: expected 12 fields, saw 16\n'
b'Skipping line 96: expected 12 fields, saw 16\n'
b'Skipping line 97: expected 12 fields, saw 16\n'
b'Skipping line 96: expected 12 fields, saw 16\n'
b'Skipping line 95: expected 12 fields, saw 16\n'
b'Skipping line 100: expected 12 fields, saw 16\

Score unavailable for file:  fuglaaXYLongstrafes - Challenge - 2020.05.15-12.10.20 Stats.csv
Score unavailable for file:  fuglaaXYLongstrafes - Challenge - 2020.05.15-12.11.27 Stats.csv
Score unavailable for file:  fuglaaXYLongstrafes - Challenge - 2020.05.15-12.12.39 Stats.csv
Score unavailable for file:  RexStrafesCata Invincible - Challenge - 2020.05.15-13.22.00 Stats.csv
Score unavailable for file:  RexStrafesCata Invincible - Challenge - 2020.05.15-13.23.06 Stats.csv
Score unavailable for file:  RexStrafesCata Invincible - Challenge - 2020.05.15-13.46.33 Stats.csv
Score unavailable for file:  RexStrafesCata Invincible - Challenge - 2020.05.15-13.47.38 Stats.csv
Score unavailable for file:  RexStrafesCata Invincible - Challenge - 2020.05.15-15.01.19 Stats.csv
Score unavailable for file:  RexStrafesCata Invincible - Challenge - 2020.05.15-15.02.24 Stats.csv
Score unavailable for file:  RexStrafesCata Invincible - Challenge - 2020.05.15-15.16.16 Stats.csv
Score unavailable for file: 

b'Skipping line 76: expected 12 fields, saw 16\n'
b'Skipping line 85: expected 12 fields, saw 16\n'
b'Skipping line 84: expected 12 fields, saw 16\n'
b'Skipping line 74: expected 12 fields, saw 16\n'
b'Skipping line 70: expected 12 fields, saw 16\n'
b'Skipping line 77: expected 12 fields, saw 16\n'
b'Skipping line 71: expected 12 fields, saw 16\n'
b'Skipping line 69: expected 12 fields, saw 16\n'
b'Skipping line 68: expected 12 fields, saw 16\n'
b'Skipping line 61: expected 12 fields, saw 16\n'
b'Skipping line 66: expected 12 fields, saw 16\n'
b'Skipping line 64: expected 12 fields, saw 16\n'
b'Skipping line 67: expected 12 fields, saw 16\n'
b'Skipping line 77: expected 12 fields, saw 16\n'
b'Skipping line 70: expected 12 fields, saw 16\n'
b'Skipping line 68: expected 12 fields, saw 16\n'
b'Skipping line 72: expected 12 fields, saw 16\n'
b'Skipping line 78: expected 12 fields, saw 16\n'
b'Skipping line 55: expected 12 fields, saw 16\n'
b'Skipping line 66: expected 12 fields, saw 16\n'


Score unavailable for file:  Vertical Long Strafes - Challenge - 2020.05.15-12.06.41 Stats.csv
Score unavailable for file:  Vertical Long Strafes - Challenge - 2020.05.15-12.07.46 Stats.csv
Score unavailable for file:  Vertical Long Strafes - Challenge - 2020.05.15-12.09.08 Stats.csv
1wall 1target
2020.05.15-12.25.26 : 73.75238
2020.05.15-12.26.31 : 73.34259
2020.05.15-13.00.20 : 67.239998
2020.05.15-13.01.50 : 65.610001
2020.05.15-12.23.14 : 426.7742
2020.05.15-12.24.20 : 497.058838
2020.05.15-12.57.49 : 431.910126
2020.05.15-12.59.05 : 445.955048
2020.05.15-13.16.32 : 432.674408
2020.05.15-13.17.37 : 473.478271
2020.05.15-13.44.23 : 470.804596
2020.05.15-13.45.27 : 355.204102
2020.05.15-14.58.17 : 392.970306
2020.05.15-14.59.21 : 445.217407
2020.05.15-15.13.30 : 446.976746
2020.05.15-15.14.34 : 510.113617
2020.05.15-15.27.58 : 474.719116
2020.05.15-15.29.02 : 413.444427
2020.05.15-15.56.00 : 556.818176
2020.05.15-15.57.06 : 499.512177
2020.05.15-15.58.10 : 632.195129
2020.05.15-15.59

2020.05.15-15.56.00 : 556.818176
2020.05.15-15.57.06 : 499.512177
2020.05.15-15.58.10 : 632.195129
2020.05.15-15.59.27 : 470.804596
2020.05.15-16.00.29 : 587.777771
2020.05.15-22.01.45 : 489.438202
2020.05.15-22.02.49 : 624.578308
2020.05.16-11.16.53 : 606.621643
2020.05.16-11.18.00 : 573.157898
2020.05.17-12.07.26 : 585.142883
2020.05.17-12.08.32 : 613.521118
2020.05.17-12.09.35 : 533.968262
2020.05.17-12.10.41 : 624.864868
2020.05.17-12.11.48 : 514.285706
2020.05.17-12.49.12 : 573.614441
2020.05.17-12.50.26 : 543.698608
2020.05.17-12.51.31 : 536.351379
2020.05.17-12.52.33 : 553.51355
2020.05.17-12.53.38 : 643.378357
2020.05.17-12.54.44 : 626.447388
2020.05.17-12.56.05 : 618.311707
2020.05.17-12.57.09 : 622.345703
2020.05.17-20.27.53 : 620.253174
2020.05.17-20.28.59 : 628.205139
2020.05.17-20.30.05 : 565.714294
2020.05.17-20.31.15 : 580.799988
2020.05.17-20.32.20 : 648.0
2020.05.18-17.00.23 : 704.390259
2020.05.18-17.01.57 : 605.568176
2020.05.18-17.04.07 : 600.119019
2020.05.18-17.05